In [3]:
import time 
from bs4 import BeautifulSoup
import requests
import pandas as pd
import random
import datetime
import re

df = pd.read_csv('daily_update.csv', index_col=0, parse_dates=True, names = ['date', 'vaccine_num_total', 'vaccine_num_daily', 'link'])
df = df.iloc[1:]
df['date'] = pd.to_datetime(df['date'])
df['date'] = df['date'].dt.date
df['vaccine_num_total'] = df['vaccine_num_total'].astype(int)
df['vaccine_num_daily'] = df['vaccine_num_daily'].astype(int)

latest_date_org = df['date'].max()
latest_vaccine_num = df[df['date'] == latest_date_org]['vaccine_num_total'].values[0]

url_cdc = 'https://www.chinacdc.cn/jkzt/crb/zl/szkb_11803/jszl_12208/'
response_cdc = requests.get(url_cdc)
response_cdc.encoding = 'utf-8'
soup_cdc = BeautifulSoup(response_cdc.text, 'html.parser')

links = soup_cdc.find_all('li')
links

max_date = datetime.date(2020, 12, 1)
max_date_link = ''
link_head = 'https://www.chinacdc.cn/jkzt/crb/zl/szkb_11803/jszl_12208/'

for i in links:
    if '新冠病毒疫苗接种情况' in i.text:
        vaccine_announce_date = i.a['href'][10:18]
        vaccine_announce_date = datetime.datetime.strptime(vaccine_announce_date, '%Y%m%d').date()
        vaccine_announce_date = vaccine_announce_date + datetime.timedelta(days=-1)

        if vaccine_announce_date > max_date:
            max_date = vaccine_announce_date
            link = i.a['href'].replace('./', link_head)
            max_date_link = link
print(max_date)
print(max_date_link)

if max_date > latest_date_org:
    url_page = max_date_link
    response_page = requests.get(url_page)
    response_page.encoding = 'utf-8'
    soup_page = BeautifulSoup(response_page.text, 'html.parser')

    page_text = soup_page.find_all(class_ = 'cn-main-detail')[0].text
    page_main_text = page_text[25:]
    page_main_text
    text_num = re.findall(r'(\d+\.\d+)', page_main_text)
    text_num = text_num[0]
    vaccine_num = int(float(text_num)*10000)
    print(vaccine_num)

    text_date = re.findall(r'(\d{4}年\d{1,2}月\d{1,2}日)', page_text)[0]
    text_date = datetime.datetime.strptime(text_date, '%Y年%m月%d日').date()
    print(text_date)

    vaccine_num_daily = vaccine_num - latest_vaccine_num
    print(vaccine_num_daily)

    df = df.append({'date': text_date, 'vaccine_num_total': vaccine_num, 'vaccine_num_daily': vaccine_num_daily, 'link': max_date_link}, ignore_index=True)
    df.to_csv('daily_update.csv')
    print('New data added')
else:
    print('No new data')

2023-01-01
https://www.chinacdc.cn/jkzt/crb/zl/szkb_11803/jszl_12208/202301/t20230102_263167.html


TypeError: expected string or bytes-like object, got 'list'

In [ ]:
url = 'https://www.chinacdc.cn/jkzt/crb/zl/szkb_11803/jszl_12208/202301/t20230102_263167.html'
response = requests.get(url)
response.encoding = 'utf-8'
soup = BeautifulSoup(response.text, 'html.parser')
soup.find_all(class_ = 'cn-main-detail')[0].text


In [ ]:

df = pd.read_csv('daily_update.csv', index_col=0, parse_dates=True, names = ['date', 'vaccine_num_total', 'vaccine_num_daily', 'link'])
df = df.iloc[1:]
# reorganize index of df
df.index = range(len(df))
df['date'] = pd.to_datetime(df['date'])
df['date'] = df['date'].dt.date
df['vaccine_num_total'] = df['vaccine_num_total'].astype(int)
df['vaccine_num_daily'] = df['vaccine_num_daily'].astype(int)

latest_date_org = df['date'].max()
date_minus_180 = latest_date_org + datetime.timedelta(days=-180)
df = df[df['date'] >= date_minus_180]

df.to_csv('daily_update_180days.csv')